In [6]:
import numpy as np
from pywarpx import picmi


constants = picmi.constants

In [7]:
from utils import cosd, sind
from space_analysis.simulation.warpx import HybridSimulation

Support `gaussian_parse_momentum_function`?


In [8]:
def init_field(
    k,
    B0,
    A,  #: relative amplitude
    theta=60,
):
    """
    Generate field with a wave propagating along the x axis at a large angle `theta` with respect to the background magnetic field lying in the x-z plane.

    The initial waveis an Alfven mode in which the magnetic field fluctuation points along the y and z axis and has a relative amplitude $A = \delta B_y / B_0$
    """

    B0x = B0 * cosd(theta)
    B0z = B0 * sind(theta)

    # dB0z = B0 * A * np.cos(k * 0)

    Bx_expression = f"{B0x}"
    By_expression = f"{A} * {B0} * cos({k} * x)"
    Bz_expression = f"{B0z}"

    return picmi.AnalyticInitialField(
        Bx_expression=Bx_expression,
        By_expression=By_expression,
        Bz_expression=Bz_expression,
    )


def init_plasma(
    vA,
    n0,
    v_ti,  #: ion thermal velocity
    k,
    B0,
    A,  #: relative amplitude
    theta=60,
):
    """
    The ion bulk and transverse velocity V remains parallel to the total transverse magnetic field.
    """
    B0x = B0 * cosd(theta)
    B0z = B0 * sind(theta)

    px_expression = vA * B0x / B0
    py_expression = f"{vA * A} * cos({k} * x)"
    pz_expression = vA * B0z / B0
    
    px_expression = 0 
    pz_expression = 0

    momentum_expressions = [px_expression, py_expression, pz_expression]

    return picmi.AnalyticDistribution(
        density_expression=n0,
        momentum_expressions=momentum_expressions,
        # rms_velocity=[v_ti] * 3,
    )

In [9]:
class AlfvenModes(HybridSimulation):
    test: bool = True
    # Applied field parameters
    dim: int = 2
    B0: float = 100 * 1e-9
    """Initial magnetic field strength (T)"""
    n0: float = 100 * 1e6
    """Initial plasma density (m^-3)"""

    vA_over_c: float = None
    """ratio of Alfven speed and the speed of light"""

    A: float = 0.1  # relative amplitude

    # Spatial domain
    Lz_norm: float = 4
    Lx_norm: float = 128  # spatial domain length in x direction (ion skin depths)

    def model_post_init(self, __context):
        """Get input parameters for the specific case desired."""
        if self.test:
            self.nppc = 64
            self.m_ion_norm = 100

        # calculate various plasma parameters based on the simulation input
        self.get_plasma_quantities()

        super().model_post_init(__context)

        self.k = 1 * 2 * np.pi / self.Lx
        self.setup_run()
        self.dump()

    def setup_field(self):
        """Setup external field"""

        B_ext = init_field(k=self.k, B0=self.B0, A=self.A)
        self._sim.add_applied_field(B_ext)
        return self

    def setup_particle(self):
        """setup the particle"""

        dist = init_plasma(
            vA=self.vA, n0=self.n0, v_ti=self.v_ti, k=self.k, B0=self.B0, A=self.A
        )

        ions = picmi.Species(
            name="ions",
            charge_state=1,
            mass=self.m_ion,
            initial_distribution=dist,
        )

        self._sim.add_species(
            ions,
            layout=picmi.PseudoRandomLayout(
                grid=self._grid, n_macroparticles_per_cell=self.nppc
            ),
        )
        return self

    def get_plasma_quantities(self):
        """Calculate various plasma parameters based on the simulation input."""
        # Ion mass (kg)
        self.m_ion = self.m_ion_norm * constants.m_e

        # Cyclotron angular frequency (rad/s) and period (s)
        self.w_ci = constants.q_e * abs(self.B0) / self.m_ion
        self.t_ci = 2.0 * np.pi / self.w_ci

        # Alfven speed (m/s): vA = B / sqrt(mu0 * n * (M + m)) = c * omega_ci / w_pi
        if self.n0 is not None:
            self.vA = self.B0 / np.sqrt(
                constants.mu0 * self.n0 * (self.m_ion + constants.m_e)
            )
        elif self.vA_over_c is not None:
            self.vA = self.vA_over_c * constants.c
            self.n0 = (self.B0 / self.vA) ** 2 / (
                constants.mu0 * (self.m_ion + constants.m_e)
            )

        # Ion plasma frequency (rad/s)
        self.w_pi = np.sqrt(constants.q_e**2 * self.n0 / (self.m_ion * constants.ep0))

        # Skin depth (m): inertial length
        self.d_i = constants.c / self.w_pi

        # Ion thermal velocity (m/s) from beta = 2 * (v_ti / vA)**2
        self.v_ti = np.sqrt(self.beta / 2.0) * self.vA

        # Temperature (eV) from thermal speed: v_ti = sqrt(kT / M)
        self.T_plasma = self.v_ti**2 * self.m_ion / constants.q_e  # eV
        self.Te = self.T_plasma

In [10]:
simulation = AlfvenModes()
simulation._sim.write_input_file()

Numerical parameters:
	dt = 5.6e-04 s
	total steps = 6400

Initializing simulation with input parameters:
	Te = 24.588 eV
	n = 1.0e+02 cm^-3
	B0 = 100.00 nT
	M/m = 100

Plasma parameters:
	d_i = 5.3e+03 m
	t_ci = 3.6e-02 s
	v_ti = 2.1e+05 m/s
	vA = 9.3e+05 m/s
	vA/c = 0.0031021956193735692



In [ ]:
simulation._sim.step()

In [9]:
simulation.diag_steps

print(f"""
    {simulation._sim.max_steps}
    {simulation._sim.time_step_size}
    """
)


    6400
    0.0005581854301372202
    


In [ ]:
# simulation._sim.step()